In [15]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Video Pose Detection

#### Testing pose tracking and live video

In [16]:
# cap = cv2.VideoCapture(0)
# ## Setup mediapipe instance
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False
      
#         # Make detection
#         results = pose.process(image)
    
#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         # Render detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

In [17]:
# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

In [18]:
# Function to determine current stage (up or down)
def current_stage(angle):
    if angle >= 130:
        return "Up"
    elif angle <= 90:
        return "Down"
    else:
        return "Mid"

In [19]:
# count = 0
# incorrect_count = 0
# posture = None
# correct_posture = True

# while cap.isOpened():
#     ret, frame = cap.read()
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(image)
    
#     if results.pose_landmarks:
#         landmarks = results.pose_landmarks.landmark
#         shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
#                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
#         elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
#                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
#         wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
#                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
#         angle = calculate_angle(shoulder, elbow, wrist)
#         stage = current_stage(angle)
        
#         if stage == "Correct":
#             incorrect_count = 0
#             correct_posture = True
#             if angle > 160:
#                 posture = "up"
#             if angle < 90 and posture == "up":
#                 posture = "down"
#         else:
#             if posture == "down" and correct_posture:
#                 incorrect_count += 1
#                 if incorrect_count == 3:
#                     print("Fix your posture! Must bend your elbows at least 90 degrees!")
#                     correct_posture = False

#         if correct_posture and posture == "down" and angle > 160:
#             count += 1
#             posture = "up"

#         cv2.putText(frame, 'Reps: {}'.format(count), 
#                     (50, 50), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
#         cv2.putText(frame, 'Stage: {}'.format(stage), 
#                     (50, 100), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
#         cv2.putText(frame, 'Posture: {}'.format(posture), 
#                     (50, 150), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
#         cv2.putText(frame, 'Angle: {:.2f}'.format(angle), 
#                     (50, 200), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
#         mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
#     cv2.imshow('Push-Up Tracker', frame)
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [20]:
# Initialize variables for counting push-ups and posture checking
counter = 0
incorrect_count = 0
posture = None
correct_posture = True

# Capture video feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Make pose detection
    results = pose.process(image)
    
    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates of shoulders, elbows, and wrists
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
        # Calculate angle
        angle = calculate_angle(shoulder, elbow, wrist)
        
        # Determine current stage (up or down)
        stage = current_stage(angle)
        
        # Check posture
        if stage == "Down":
            if angle >= 90:
                incorrect_count = 0
                correct_posture = True
            else:
                incorrect_count += 1
                if incorrect_count == 3:
                    cv2.putText(image, "Fix your posture! Must bend your elbows at least 90 degrees!", 
                                (50, 100), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
                    correct_posture = False
        elif stage == "Up":
            if correct_posture:
                counter += 1
                correct_posture = False
        
        # Render push-up counter and feedback
        cv2.putText(image, 'Reps: {}'.format(counter), 
                    (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Stage: {}'.format(stage), 
                    (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Posture: {}'.format(posture), 
                    (50, 150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Angle: {:.2f}'.format(angle), 
                    (50, 200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
    
    except:
        pass
    
    # Display the output
    cv2.imshow('Push-Up Tracker', image)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()